In [27]:
import json
import pandas as pd
import folium

Pull complete map from https://geo.nyu.edu/
U.S. Neighborhoods greenness measures and social variables

Load as json and pull the cities that are needed for data set

In [3]:
with open('stanford-xq082nw3443-geojson.json') as json_data:
    all_data = json.load(json_data)

# cut down the data
neighborhoods_data = all_data['features']

In [71]:
neighborhoods_data[-1]

{'type': 'Feature',
 'id': 'xq082nw3443.3155',
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-104.8852777, 39.70742466],
     [-104.8860689, 39.70742466],
     [-104.88686011, 39.70742466],
     [-104.89160733, 39.71058947],
     [-104.89239854, 39.71138068],
     [-104.89239854, 39.71138068],
     [-104.89318974, 39.71138068],
     [-104.9042666, 39.71138068],
     [-104.90664021, 39.71138068],
     [-104.91059623, 39.71138068],
     [-104.90743141, 39.71454549],
     [-104.90980503, 39.71454549],
     [-104.91296984, 39.71454549],
     [-104.91296984, 39.71850152],
     [-104.91296984, 39.71850152],
     [-104.90822262, 39.71850152],
     [-104.90822262, 39.71850152],
     [-104.90822262, 39.71850152],
     [-104.90822262, 39.72166633],
     [-104.90822262, 39.72166633],
     [-104.90664021, 39.72166633],
     [-104.90347539, 39.72166633],
     [-104.90347539, 39.72245754],
     [-104.90347539, 39.72324874],
     [-104.90347539, 39.72483115],
     [-104.90347539, 39.7256

In [18]:
target_cities = ['Denver', 'Seattle', 'Austin', 'Portland', 'Phoenix']

In [73]:
# define the dataframe columns
column_names = ['City', 'State','Neighborhood', 'Latitude', 'Longitude', 'Home Ownership',
       'Home Rentership', 'Home Value', 'Median Age', 'Population Denisty',
       'Boundary Box'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    if data['properties']['city'] not in target_cities:
        pass
    else:
        city =  data['properties']['city'] 
        state = data['properties']['state']
        neighborhood_name = data['properties']['name']
        neighborhood_lat = data['properties']['x']
        neighborhood_lon = data['properties']['y']
        home_val = data['properties']['avg_hval']
        home_own = data['properties']['pct_own']
        home_rent = data['properties']['pct_rent']
        median_age = data['properties']['medage_cy']
        pop_density = data['properties']['popdensity']
        boundary_box = data['geometry']
        neighborhoods = neighborhoods.append({'City': city,
                                              'State': state,
                                              'Neighborhood': neighborhood_name,
                                              'Latitude': neighborhood_lat,
                                              'Longitude': neighborhood_lon,
                                              'Home Value': home_val,
                                              'Home Ownership': home_own,
                                              'Home Rentership': home_rent,
                                              'Median Age': median_age,
                                              'Population Denisty': pop_density,
                                              'Boundary Box': boundary_box
                                             }, ignore_index=True)

save dataframe to csv for use

In [96]:
neighborhoods.to_csv('neighborhoods.csv', index = False)

How well do the boundaries work overlaid on the map. Look at the last neighborhood in our list

https://python-visualization.github.io/folium/quickstart.html#Choropleth-maps

In [74]:
test_geo = {"type":"FeatureCollection",
            "features":[
                {"type":
                 "Feature",
                 "id": data['properties']['state'],
                 "properties":{
                     "name":data['properties']['name']}
                 ,"geometry": data['geometry']}]}

In [75]:
lat = data['properties']['x']
long = data['properties']['y']

m = folium.Map(location = [long, lat], zoom_start = 12)

folium.Choropleth(
    geo_data = test_geo
    
).add_to(m)
m

create new json files for each of the city's and their neighborhood boundaries

In [85]:
for city in neighborhoods.City.unique():
    temp_df = neighborhoods[neighborhoods['City'] == city]
    geo_json = {"type":"FeatureCollection",
            "features":[]}
    for i, row in temp_df.iterrows():
        geo_json['features'].append(
                {"type":
                 "Feature",
                 "id": row['Neighborhood'],
                 "properties":{
                     "name": row['Neighborhood']}
                 ,"geometry": row['Boundary Box']})


    with open(f'{city}_geo.json', 'w') as fp:
        json.dump(geo_json, fp)

Checkout how those new jsons look

In [95]:
long = 39.70742466
lat = -104.8852777 

# playing with the tiles. I may have a mapbox api key to get more options
m = folium.Map(location = [long, lat], zoom_start = 11, tiles = 'Stamen Toner') 

folium.Choropleth(
    geo_data = geo_json,
    data = temp_df,
    columns = ['Neighborhood', 'Home Ownership'],
    key_on = 'feature.id',
    fill_color = 'YlGn',
    line_opacty = 0.2,
    legend_name = 'Home Ownership'
    
).add_to(m)

folium.LayerControl().add_to(m)

m